### OLS 회귀분석

In [17453]:
### 라이브러리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

from xgboost import XGBRegressor

from scipy import stats
from scipy.stats import shapiro

import statsmodels.api as sm
from statsmodels.stats.stattools import jarque_bera
from statsmodels.stats.diagnostic import het_breuschpagan
from scipy.stats import shapiro

### 환경설정
np.random.seed(4)


### 함수 정의
def preprocess_transaction_flag(df):
    df['transaction_flag'] = df['transaction_count'].apply(lambda x: 1 if x > 0 else 0)
    return df

def trim_date(df, start, end):
    ### df date 형식 맞춰주기
    df['date'] = df['date'].astype(str)
    if len(df['date'][0]) == 8:
        df['date'] = df['date'].apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:])


    return df[(df['date']>=start) & (df['date']<=end)]

In [17454]:
price_df = pd.read_csv('./data/price/bitcoin_price_coin_market_cap.csv').set_index('date')
price_df

,open,high,low,close,volume,marketCap
date,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,8.036550e+06,4.297536e+09
2015-01-02,314.079010,315.838989,313.565002,315.032013,7.860650e+06,4.309551e+09
2015-01-03,314.846008,315.149994,281.082001,281.082001,3.305440e+07,3.846270e+09
2015-01-04,281.145996,287.230011,257.612000,264.195007,5.562910e+07,3.616321e+09
2015-01-05,265.084015,278.341003,265.084015,274.473999,4.396280e+07,3.758098e+09
...,...,...,...,...,...,...
2025-03-08,86742.652990,86847.267594,85247.484188,86154.593210,1.820612e+10,1.708771e+12
2025-03-09,86154.305891,86471.130163,80052.486979,80601.041311,3.089935e+10,1.598205e+12
2025-03-10,80597.149784,83955.926252,77420.592186,78532.001808,5.406110e+10,1.558444e+12


### 데이터 세팅

In [17455]:
# ### 학습용 데이터 읽기
# test_df = pd.read_csv("./data/test/test_df.csv")
# test_df.set_index('date', inplace=True)
### 학습용 데이터 읽기
test_df = pd.read_csv("./data/test/test_df.csv")
test_df.set_index('date', inplace=True)
test_df

magazine_pro = pd.read_csv("./data/test/magazine_preprocessed_index.csv").set_index('date')

test_df = test_df.merge(magazine_pro, left_index=True, right_index=True, how='left')

# test_df['close'] = price_df['close']
# test_df['open'] = price_df['open']
# test_df['high'] = price_df['high']
# test_df['low'] = price_df['low']
# test_df['volume'] = price_df['volume']

test_df['returns'] = test_df['close'].pct_change()
test_df['returns_during_2'] = test_df['close'].pct_change(2)
test_df['returns_during_3'] = test_df['close'].pct_change(3)
test_df['returns_during_4'] = test_df['close'].pct_change(4)
test_df['returns_during_5'] = test_df['close'].pct_change(5)
test_df['returns_during_6'] = test_df['close'].pct_change(6)
test_df['returns_during_7'] = test_df['close'].pct_change(7)
test_df['returns_during_8'] = test_df['close'].pct_change(8)
test_df['returns_during_9'] = test_df['close'].pct_change(9)
test_df['returns_during_10'] = test_df['close'].pct_change(10)
test_df['returns_during_11'] = test_df['close'].pct_change(11)
test_df['returns_during_12'] = test_df['close'].pct_change(12)
test_df['returns_during_13'] = test_df['close'].pct_change(13)
test_df['returns_during_14'] = test_df['close'].pct_change(14)
test_df['returns_during_15'] = test_df['close'].pct_change(15)
test_df['returns_during_16'] = test_df['close'].pct_change(16)
test_df['returns_during_17'] = test_df['close'].pct_change(17)
test_df['returns_during_18'] = test_df['close'].pct_change(18)
test_df['returns_during_19'] = test_df['close'].pct_change(19)
test_df['returns_during_20'] = test_df['close'].pct_change(20)



### Realized Volatility 추가
test_df['rv'] = test_df['returns'].rolling(5).std()
test_df['rv'] = test_df['rv'].fillna(test_df['rv'].mean())

### 이벤트용 데이터 읽기
event_df = pd.read_csv("./data/event/final/bitcoin_event_details_sentiment_250324.csv")
event_df.index = event_df['Date'].astype(str).apply(lambda x: x[0:4]+'-'+x[4:6]+'-'+x[6:])
# event_df = event_df[event_df['predict']==1]

target_event_receive_df = pd.read_csv("./data/target/final/event_2_day_transactions_over_120_threshold_1000_receive.csv")
# target_event_receive_df = pd.read_csv("./data/target/final/without_conference_event_2_day_transactions_over_40_threshold_1000_from_2018_receive.csv")
# target_event_receive_df = pd.read_csv("./data/target/screened_event_2_day_transactions_over_20_threshold_500_from_2018_receive.csv")
# target_event_receive_df = pd.read_csv("./data/target/without_conference_event_2_day_transactions_over_40_threshold_500_from_2018_receive.csv")

target_event_receive_df = preprocess_transaction_flag(target_event_receive_df)
target_event_receive_df.columns = ['date', 'transaction_count', 'transaction_amount', 'transaction_flag']

target_event_receive_df = trim_date(target_event_receive_df, (test_df.index)[0], (test_df.index)[-1])


### Bitcoin Event Flag 추가 / 긍정, 부정, 애매 이벤트 필터링
# pos, neg, neu, all
event_sentiment = 'all'
if event_sentiment == 'all':
    pass
elif event_sentiment == 'pos':
    event_df = event_df[event_df['classification']==1]
elif event_sentiment == 'neg':
    event_df = event_df[event_df['classification']==0]
elif event_sentiment == 'neu':
    event_df = event_df[event_df['classification']==2]
event_df['classification'] = 1
event_df = event_df['classification']
event_df = event_df[~event_df.index.duplicated(keep='first')]

test_df['event_flag'] = event_df
test_df['event_flag'] = test_df['event_flag'].fillna(0)
event_flag_copy = test_df['event_flag'].values

test_df['big_returns_flag'] = test_df['abs_returns'].apply(lambda x: 1 if x > 0.05 else 0)
big_returns_flag_copy = test_df['big_returns_flag'].values

### 분석 데이터 선택
target_df = target_event_receive_df.copy().set_index('date')

### test_df에 target_df 데이터 적용
test_df['transaction_count'] = target_df['transaction_count']
test_df['transaction_amount'] = target_df['transaction_amount']
test_df['transaction_flag'] = target_df['transaction_flag']
test_df['transaction_amount_usd'] = test_df['transaction_amount'] * test_df['close']

### test_df의 target 선택
test_df['target'] = test_df['transaction_amount_usd']
test_df['target'] = np.where(test_df['target'] == 0, 1e-10, test_df['target'])

### target 관련 파생변수 생성
test_df['target_delta'] = test_df['target'].diff(1)
test_df['target_returns'] = test_df['target'].pct_change(1)
test_df['target_flag'] = test_df['target_delta'].apply(lambda x: 1 if x > 0 else 0)
target_flag_copy = test_df['target_flag'].values

### VPIN 관련 변수 추가
vpin = pd.read_csv('./data/vpin/vpin.csv')
vpin['ma_10'] = vpin['vpin'].rolling(10).mean()
vpin['ma_10'] = vpin['ma_10'].fillna(vpin['ma_10'].mean())
vpin = vpin.set_index('date')

test_df['vpin'] = vpin['vpin']
test_df['vpin_ma_10'] = vpin['ma_10']

### 회귀분석 돌릴때 Cheating 방지용으로 transaction 관련 target값 삭제
transaction_count = test_df['transaction_count'].copy()
transaction_amount = test_df['transaction_amount'].copy()
transaction_amount_usd = test_df['transaction_amount_usd'].copy()

### 수익률 관련 변수
returns_copy = test_df['returns'].copy()

##### 전처리 관련
### 변화율에서 inf값 나올텐데 그거 어떻게 처리할지
test_df = test_df.replace([np.inf, -np.inf], np.nan)

### NaN값 처리 방법
test_df = test_df.fillna(test_df.mean())

### 스케일링
scaler = StandardScaler()
test_df = pd.DataFrame(scaler.fit_transform(test_df), columns=test_df.columns, index=test_df.index)

### 스케일링 유의 변수
test_df['returns'] = returns_copy
test_df['event_flag'] = event_flag_copy
test_df['big_returns_flag'] = big_returns_flag_copy
test_df['target_flag'] = target_flag_copy

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2201096175.py:19: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  test_df['returns'] = test_df['close'].pct_change()
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2201096175.py:20: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  test_df['returns_during_2'] = test_df['close'].pct_change(2)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2201096175.py:21: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or s

In [17456]:
target_event_receive_df

,date,transaction_count,transaction_amount,transaction_flag
0,2017-01-01,5,10000.0,1
1,2017-01-02,0,0.0,0
2,2017-01-03,0,0.0,0
3,2017-01-04,5,7500.0,1
4,2017-01-05,0,0.0,0
...,...,...,...,...
2825,2024-09-26,0,0.0,0
2826,2024-09-27,0,0.0,0
2827,2024-09-28,2,6400.0,1
2828,2024-09-29,0,0.0,0


In [17457]:
pd.set_option('display.max_columns', None)
test_df

,transaction_count,transaction_amount,transaction_flag,returns,close,abs_returns,volume,open,high,low,high_low,daily_avg_trades,daily_sum_trades,daily_sum_taker_buy_base_asset_volume,daily_sum_taker_buy_quote_asset_volume,daily_avg_spread,daily_quote_asset_volume,daily_avg_quote_asset_volume,daily_low,daily_high,daily_avg_price,daily_vwap,daily_avg_volume,daily_morning_returns,daily_afternoon_returns,daily_night_returns,daily_dawn_returns,daily_avg_morning_returns,daily_avg_afternoon_returns,daily_avg_night_returns,daily_avg_dawn_returns,daily_avg_morning_volume,daily_avg_afternoon_volume,daily_avg_night_volume,daily_avg_dawn_volume,daily_morning_volatility,daily_afternoon_volatility,daily_night_volatility,daily_dawn_volatility,daily_volatility,avg-block-size,blocks-size,cost-per-transaction-percent,cost-per-transaction,difficulty,estimated-transaction-volume-usd,estimated-transaction-volume,fees-usd-per-transaction,hash-rate,market-price,median-confirmation-time,miners-revenue,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,trade-volume,transaction-fees-usd,transaction-fees,n-payments-per-block,n-payments,avg-confirmation-time,10Yr+ HODL Wave,1Yr+ HODL Wave,5Yr+ HODL Wave,Addresses with > 0 BTC,Addresses with > 0.01 BTC,Addresses with > 0.1 BTC,Addresses with > 1 BTC,Addresses with > 10 BTC,Addresses with > 100 BTC,"Addresses with > 1,000 BTC","Addresses with > 10,000 BTC",Balanced Price,Active Addresses 7DMA,Hash Rate 7DMA,Miner Difficulty,Circulating Supply,CDD (90dma),CVDD,Delta Top,Long-Term Holder Realized Price,Miner Revenue - Block Rewards,Miner Revenue - Fees,Miner Fees %,Miner Revenue - Total,New Addresses,Addresses in Loss,Addresses in Profit,Realized Price,MVOCD,Short-Term Holder Realized Price,Supply Adjusted CDD (90dma),Terminal Price,Top Cap,VDD Multiple,Whale Shadows,returns_during_2,returns_during_3,returns_during_4,returns_during_5,returns_during_6,returns_during_7,returns_during_8,returns_during_9,returns_during_10,returns_during_11,returns_during_12,returns_during_13,returns_during_14,returns_during_15,returns_during_16,returns_during_17,returns_during_18,returns_during_19,returns_during_20,rv,event_flag,big_returns_flag,transaction_amount_usd,target,target_delta,target_returns,target_flag,vpin,vpin_ma_10
date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-01-01,-0.296743,-0.183598,0.894569,NaN,-1.111536,0.000000,-0.880727,-1.112165,-1.107929,-1.107056,-0.752190,-1.669788e-16,0.000000,0.000000,0.000000,-6.677695e-17,0.000000,0.000000,0.000000,1.906936e-16,0.000000,0.000000,0.000000,-1.616865e-18,0.000000,6.472890e-18,-6.366045e-18,-1.438860e-18,4.067258e-18,2.330567e-18,0.000000,0.000000,0.000000,-1.219491e-16,-1.393576e-16,0.000000,-1.384638e-16,-1.385767e-16,0.000000,-1.318389e-16,6.953688e-16,0.000000,3.645692e-16,0.000000,0.000000,1.936683e-16,0.000000,0.000000,1.652036e-16,0.000000,-5.403927e-16,2.357973e-16,-5.153963e-16,0.000000,-4.867979e-16,-5.204247e-16,-1.099428e-15,-1.524501e-16,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.391100,-0.004522,-1.283663,-1.827698,-1.924960,-1.896433,-2.052649,-1.119430,2.277760,-1.345102,0.596044,-1.329585,-1.582273,-1.011193,-1.010653,-2.076837,-0.621870,-1.235382,-1.360063,-1.203051,1.212060,-0.149535,-0.373813,1.080115,-2.249975,-1.468576,1.468576,-1.329661,-0.880017,-1.153263,-0.341108,-1.277422,-1.173342,0.442407,-0.092031,-1.736282e-17,0.000000,0.000000,0.000000,-1.942717e-17,0.000000,-1.647062e-17,0.000000,0.000000,0.000000,0.000000,0.000000,-4.740562e-17,2.269673e-17,2.180099e-17,-2.100669e-17,0.000000,3.916300e-17,0.000000,0.000000,0.0,0,-0.349973,-0.349973,-5.403487e-19,-1.521418e-17,0,1.169821,-0.006735
2017-01-02,-0.313983,-0.194081,-1.117856,-0.041238,-1.121003,0.653487,-0.875523,-1.110900,-1.113896,-1.121457,-0.640992,-1.669788e-16,0.000000,0.000000,0.000000,-6.677695e-17,0.0000

### Target의 Returns예측

In [17458]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target', 'target_returns']].fillna(test_df.mean())
X['target_returns'] = X['target_returns'].shift(-1).fillna(0)
# X = X[['target']]

# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     6.956
Date:                Mon, 07 Apr 2025   Prob (F-statistic):           0.000972
Time:                        16:58:05   Log-Likelihood:                 4696.4
No. Observations:                2465   AIC:                            -9387.
Df Residuals:                    2462   BIC:                            -9369.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------


### During 2 day returns 예측

In [17459]:
# Independent, Dependent 분리
y = test_df['returns_during_2'].shift(-2).fillna(test_df['returns_during_2'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_2   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1056
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.745
Time:                        16:58:05   Log-Likelihood:                -3473.1
No. Observations:                2465   AIC:                             6950.
Df Residuals:                    2463   BIC:                             6962.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 3 day 예측

In [17460]:
# Independent, Dependent 분리
y = test_df['returns_during_3'].shift(-3).fillna(test_df['returns_during_3'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_3   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.2193
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.640
Time:                        16:58:05   Log-Likelihood:                -3469.8
No. Observations:                2465   AIC:                             6944.
Df Residuals:                    2463   BIC:                             6955.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 4 day 예측

In [17461]:
# Independent, Dependent 분리
y = test_df['returns_during_4'].shift(-4).fillna(test_df['returns_during_4'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_4   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.751
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.186
Time:                        16:58:05   Log-Likelihood:                -3467.1
No. Observations:                2465   AIC:                             6938.
Df Residuals:                    2463   BIC:                             6950.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 5 day 예측

In [17462]:
# Independent, Dependent 분리
y = test_df['returns_during_5'].shift(-5).fillna(test_df['returns_during_5'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_5   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     8.780
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00307
Time:                        16:58:05   Log-Likelihood:                -3465.2
No. Observations:                2465   AIC:                             6934.
Df Residuals:                    2463   BIC:                             6946.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 6 day 예측

In [17463]:
# Independent, Dependent 분리
y = test_df['returns_during_6'].shift(-6).fillna(test_df['returns_during_6'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_6   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     14.22
Date:                Mon, 07 Apr 2025   Prob (F-statistic):           0.000166
Time:                        16:58:05   Log-Likelihood:                -3458.2
No. Observations:                2465   AIC:                             6920.
Df Residuals:                    2463   BIC:                             6932.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 7 day 예측

In [17464]:
# Independent, Dependent 분리
y = test_df['returns_during_7'].shift(-7).fillna(test_df['returns_during_7'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_7   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     13.79
Date:                Mon, 07 Apr 2025   Prob (F-statistic):           0.000209
Time:                        16:58:05   Log-Likelihood:                -3451.4
No. Observations:                2465   AIC:                             6907.
Df Residuals:                    2463   BIC:                             6918.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 8 day 예측

In [17465]:
# Independent, Dependent 분리
y = test_df['returns_during_8'].shift(-8).fillna(test_df['returns_during_8'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_8   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     10.34
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00132
Time:                        16:58:05   Log-Likelihood:                -3447.2
No. Observations:                2465   AIC:                             6898.
Df Residuals:                    2463   BIC:                             6910.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 9 Days

In [17466]:
# Independent, Dependent 분리
y = test_df['returns_during_9'].shift(-9).fillna(test_df['returns_during_9'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:       returns_during_9   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     11.46
Date:                Mon, 07 Apr 2025   Prob (F-statistic):           0.000721
Time:                        16:58:05   Log-Likelihood:                -3440.6
No. Observations:                2465   AIC:                             6885.
Df Residuals:                    2463   BIC:                             6897.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### During 10 Days

In [17467]:
# Independent, Dependent 분리
y = test_df['returns_during_10'].shift(-10).fillna(test_df['returns_during_10'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_10   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     9.935
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00164
Time:                        16:58:05   Log-Likelihood:                -3434.2
No. Observations:                2465   AIC:                             6872.
Df Residuals:                    2463   BIC:                             6884.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17468]:
# Independent, Dependent 분리
y = test_df['returns_during_11'].shift(-11).fillna(test_df['returns_during_11'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_11   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.363
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00670
Time:                        16:58:05   Log-Likelihood:                -3422.5
No. Observations:                2465   AIC:                             6849.
Df Residuals:                    2463   BIC:                             6861.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17469]:
# Independent, Dependent 분리
y = test_df['returns_during_12'].shift(-12).fillna(test_df['returns_during_12'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===


                            OLS Regression Results                            
Dep. Variable:      returns_during_12   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     6.455
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0111
Time:                        16:58:05   Log-Likelihood:                -3409.8
No. Observations:                2465   AIC:                             6824.
Df Residuals:                    2463   BIC:                             6835.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0346      0.054     -0.636      0.5

In [17470]:
# Independent, Dependent 분리
y = test_df['returns_during_13'].shift(-13).fillna(test_df['returns_during_13'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_13   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     6.222
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0127
Time:                        16:58:05   Log-Likelihood:                -3394.1
No. Observations:                2465   AIC:                             6792.
Df Residuals:                    2463   BIC:                             6804.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17471]:
# Independent, Dependent 분리
y = test_df['returns_during_14'].shift(-14).fillna(test_df['returns_during_14'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_14   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     5.982
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0145
Time:                        16:58:05   Log-Likelihood:                -3381.6
No. Observations:                2465   AIC:                             6767.
Df Residuals:                    2463   BIC:                             6779.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17472]:
# Independent, Dependent 분리
y = test_df['returns_during_15'].shift(-15).fillna(test_df['returns_during_15'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_15   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     6.109
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0135
Time:                        16:58:06   Log-Likelihood:                -3374.1
No. Observations:                2465   AIC:                             6752.
Df Residuals:                    2463   BIC:                             6764.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17473]:
# Independent, Dependent 분리
y = test_df['returns_during_16'].shift(-16).fillna(test_df['returns_during_16'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_16   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     7.789
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00530
Time:                        16:58:06   Log-Likelihood:                -3364.4
No. Observations:                2465   AIC:                             6733.
Df Residuals:                    2463   BIC:                             6744.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17474]:
# Independent, Dependent 분리
y = test_df['returns_during_17'].shift(-17).fillna(test_df['returns_during_17'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_17   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     8.046
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00460
Time:                        16:58:06   Log-Likelihood:                -3359.2
No. Observations:                2465   AIC:                             6722.
Df Residuals:                    2463   BIC:                             6734.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17475]:
# Independent, Dependent 분리
y = test_df['returns_during_18'].shift(-18).fillna(test_df['returns_during_18'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_18   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     7.356
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00673
Time:                        16:58:06   Log-Likelihood:                -3353.0
No. Observations:                2465   AIC:                             6710.
Df Residuals:                    2463   BIC:                             6722.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17476]:
# Independent, Dependent 분리
y = test_df['returns_during_19'].shift(-19).fillna(test_df['returns_during_19'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_19   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     7.570
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00598
Time:                        16:58:06   Log-Likelihood:                -3344.9
No. Observations:                2465   AIC:                             6694.
Df Residuals:                    2463   BIC:                             6705.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

In [17477]:
# Independent, Dependent 분리
y = test_df['returns_during_20'].shift(-20).fillna(test_df['returns_during_20'].mean())
X = test_df[['target']].fillna(test_df.target.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:      returns_during_20   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     9.433
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00215
Time:                        16:58:06   Log-Likelihood:                -3336.6
No. Observations:                2465   AIC:                             6677.
Df Residuals:                    2463   BIC:                             6689.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### Event Flag를 더미변수로 추가하면 기울기가 늘어나는지

In [17478]:
# 파생변수 추가
test_df['event_target'] = test_df['target'] * test_df['event_flag'].shift(0).fillna(0)

# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['target', 'volume', 'event_flag']]
X['event_flag'] = X['event_flag'].shift(-2).fillna(0)
X['target_event'] = (X['target'] * X['event_flag']).shift(0).fillna(0)
X = X[['target_event']]

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 3})
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1959
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.658
Time:                        16:58:06   Log-Likelihood:                 4691.3
No. Observations:                2465   AIC:                            -9379.
Df Residuals:                    2463   BIC:                            -9367.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
cons

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2558841529.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['event_flag'] = X['event_flag'].shift(-2).fillna(0)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2558841529.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_event'] = (X['target'] * X['event_flag']).shift(0).fillna(0)


### Big Returns Flag 추가 설명력 확인

In [17479]:
# 파생변수 추가
test_df['event_target'] = test_df['target'] * test_df['event_flag'].shift(0).fillna(0)

# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target', 'volume', 'vpin', 'big_returns_flag']]
X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
X['target_big_returns'] = X['target'] * X['big_returns_flag']
X = X[['target_big_returns']]

# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     7.591
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00591
Time:                        16:58:06   Log-Likelihood:                 4697.5
No. Observations:                2465   AIC:                            -9391.
Df Residuals:                    2463   BIC:                            -9379.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\636899221.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\636899221.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_big_returns'] = X['target'] * X['big_returns_flag']


### 과거 변동성이 Target을 잘 설명하는지

In [17480]:
# test_df['volatility'] = test_df['returns']**2
# test_df['std_5'] = test_df['returns'].rolling(5).std()
# test_df['volume_5'] = test_df['volume'].rolling(5).std()

# # Independent, Dependent 분리
# y = test_df['target']
# X = test_df[['volume_5', 'std_5']].fillna(test_df.mean())

# # ## train, test 데이터 분리
# # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.35, shuffle=False)

# start = '2018-01-01'
# end = '2024-12-31'
# X_test = X[(X.index >= start)&(X.index <= end)]
# y_test = y[(y.index >= start)&(y.index <= end)]

# X_with_const = sm.add_constant(X_test)

# # OLS 회귀 적합
# # ---------------------------------------------
# lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 3})
# print("=== OLS with Newey-West Standard Errors ===")
# print(lr.summary())

### Target Flag가 Returns를 유의미하게 설명하는지

In [17481]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['vpin', 'target_flag', 'returns']].fillna(test_df.mean())
# X['target_flag'] = X['target_delta'].apply(lambda x: 1 if x > 0.05 else 0)
# X['target_flag'] = X['target_flag'] * X['target_delta']
X = X[['target_flag']]

## train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.813
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0936
Time:                        16:58:06   Log-Likelihood:                 4692.6
No. Observations:                2465   AIC:                            -9381.
Df Residuals:                    2463   BIC:                            -9370.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
const 

### Target의 변화율이 Returns를 유의미하게 설명하는지

In [17482]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target']].fillna(test_df.mean())


# # train, test 데이터 분리
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle=False)

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin'].shift(1)

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.150
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.143
Time:                        16:58:06   Log-Likelihood:                 4693.0
No. Observations:                2465   AIC:                            -9382.
Df Residuals:                    2463   BIC:                            -9370.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

### Target 변화량, 변화율, 값으로 Returns 예측

In [17483]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target', 'target_returns']]
X['target_returns'] = X['target_returns'].shift(-1).fillna(X['target_returns'].mean())
X = X[['target']]


start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.150
Date:                Mon, 07 Apr 2025   Prob (F-statistic):              0.143
Time:                        16:58:06   Log-Likelihood:                 4693.0
No. Observations:                2465   AIC:                            -9382.
Df Residuals:                    2463   BIC:                            -9370.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const   

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2454022158.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_returns'] = X['target_returns'].shift(-1).fillna(X['target_returns'].mean())


In [17484]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'event_flag' ,'big_returns_flag']]
X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
X['target_event'] = X['target_returns'] * X['big_returns_flag']
X['target_event'] = X['target_event'].shift(0).fillna(0)
X = X[['target_event']]


start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     6.508
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0108
Time:                        16:58:06   Log-Likelihood:                 4695.1
No. Observations:                2465   AIC:                            -9386.
Df Residuals:                    2463   BIC:                            -9374.
Df Model:                           1                                         
Covariance Type:                  HAC                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
cons

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2946209041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2946209041.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_event'] = X['target_returns'] * X['big_returns_flag']
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2946209041.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

### Control Not Rational

In [17485]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'target', 'event_flag', 'big_returns_flag', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
X['target_event'] = X['target'] * X['event_flag']
X['target_event'] = X['target_event'].shift(-1).fillna(0)

X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
X['target_big_returns'] = X['target'] * X['big_returns_flag']

X['target_returns'] = X['target_returns'].shift(-1).fillna(test_df['target_returns'].mean())

X['vpin'] = X['vpin'].shift(0).fillna(X['vpin'].mean())
X['returns'] = X['returns'].shift(0).fillna(X['returns'].mean())
X['volume'] = X['volume'].shift(0).fillna(X['volume'].mean())
X['rv'] = X['rv'].shift(0).fillna(X['rv'].mean())
X['n-transactions'] = X['n-transactions'].shift(0).fillna(X['n-transactions'].mean())
X = X[['target', 'target_returns', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     2.566
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0124
Time:                        16:58:06   Log-Likelihood:                 4700.1
No. Observations:                2465   AIC:                            -9384.
Df Residuals:                    2457   BIC:                            -9338.
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------


C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3045581581.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3045581581.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_event'] = X['target'] * X['event_flag']
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3045581581.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

### Control Rational Lag

In [17486]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-4).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'target', 'event_flag', 'big_returns_flag', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
X['target_event'] = X['target'] * X['event_flag']
X['target_event'] = X['target_event'].shift(-1).fillna(0)

X['big_returns_flag'] = X['big_returns_flag'].shift(-4).fillna(0)
X['target_big_returns'] = X['target'] * X['big_returns_flag']
X['target_big_returns'] = X['target_big_returns'].shift(0).fillna(0)
# X['target_event'] = X['target_event'].shift(0).fillna(0)

X['target_returns'] = X['target_returns'].shift(-1).fillna(test_df['target_returns'].mean())
# X['target'] = X['target'].shift(1).fillna(test_df['target'].mean())

X['vpin'] = X['vpin'].shift(-3).fillna(X['vpin'].mean())
X['returns'] = X['returns'].shift(-3).fillna(X['returns'].mean())
X['volume'] = X['volume'].shift(-3).fillna(X['volume'].mean())
X['rv'] = X['rv'].shift(-3).fillna(X['rv'].mean())
X['n-transactions'] = X['n-transactions'].shift(-3).fillna(X['n-transactions'].mean())
X = X[['target_returns', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]

start = '2018-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     2.558
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0180
Time:                        16:58:06   Log-Likelihood:                 4701.6
No. Observations:                2465   AIC:                            -9389.
Df Residuals:                    2458   BIC:                            -9349.
Df Model:                           6                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------


C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\1868175168.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\1868175168.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_event'] = X['target'] * X['event_flag']
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\1868175168.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [17487]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-5).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'target', 'event_flag', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
X['target_returns'] = X['target_returns'].shift(-2).fillna(method='ffill')
X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
X['target_event'] = X['target'] * X['event_flag']
X['target_event'] = X['target_event'].shift(-2).fillna(0)
X['vpin'] = X['vpin'].shift(-1).fillna(X['vpin'].mean())
X['returns'] = X['returns'].shift(-4).fillna(X['returns'].mean())
X['volume'] = X['volume'].shift(-4).fillna(X['volume'].mean())
X['rv'] = X['rv'].shift(-1).fillna(X['rv'].mean())
X['target'] = X['target'].shift(0).fillna(X['target'].mean())
X['n-transactions'] = X['n-transactions'].shift(-2).fillna(X['n-transactions'].mean())
X = X[['target_event', 'target_returns', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
# , , 'vpin', 'volume', 'returns', 'rv',
start = '2023-01-01'
end = '2023-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     2.309
Date:                Mon, 07 Apr 2025   Prob (F-statistic):             0.0259
Time:                        16:58:06   Log-Likelihood:                 859.30
No. Observations:                 365   AIC:                            -1703.
Df Residuals:                     357   BIC:                            -1671.
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------


C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2273610423.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X['target_returns'] = X['target_returns'].shift(-2).fillna(method='ffill')
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2273610423.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_returns'] = X['target_returns'].shift(-2).fillna(method='ffill')
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\2273610423.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

In [17488]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'target', 'big_returns_flag', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
X['target_returns'] = X['target_returns'].shift(0).fillna(method='ffill')
X['big_returns_flag'] = X['big_returns_flag'].shift(-3).fillna(0)
X['target'] = X['target'].shift(0).fillna(X['target'].mean())
X['target_big_returns'] = X['target'] * X['big_returns_flag']
X['target_big_returns'] = X['target_big_returns'].shift(0).fillna(0)
X['vpin'] = X['vpin'].shift(-2).fillna(X['vpin'].mean())
X['returns'] = X['returns'].shift(-2).fillna(X['returns'].mean())
X['volume'] = X['volume'].shift(-2).fillna(X['volume'].mean())
X['rv'] = X['rv'].shift(-2).fillna(X['rv'].mean())
# X['target'] = X['target'].shift(2).fillna(X['target'].mean())
X['n-transactions'] = X['n-transactions'].shift(-1).fillna(X['n-transactions'].mean())
X = X[['target_returns', 'target_big_returns', 'vpin', 'volume', 'returns', 'rv', 'n-transactions']]
# , 'target_event', 'vpin', 'volume', 'returns', 'rv'
start = '2024-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

# X_test['vpin'] = test_df['vpin']
# X_test = X_test['vpin']

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
# lr = sm.OLS(y_test, X_with_const).fit()
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.039
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.461
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00145
Time:                        16:58:06   Log-Likelihood:                 595.18
No. Observations:                 274   AIC:                            -1174.
Df Residuals:                     266   BIC:                            -1145.
Df Model:                           7                                         
Covariance Type:                  HAC                                         
                         coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3829531229.py:4: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  X['target_returns'] = X['target_returns'].shift(0).fillna(method='ffill')
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3829531229.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_returns'] = X['target_returns'].shift(0).fillna(method='ffill')
C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\3829531229.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### Event Flag 추가 Target 3종세트 Returns 예측

In [17489]:
# Independent, Dependent 분리
y = test_df['returns'].shift(-3).fillna(test_df['returns'].mean())
X = test_df[['target_returns', 'target', 'event_flag']]
X['event_flag'] = X['event_flag'].shift(-3).fillna(0)
X['target_event'] = X['target'] * X['event_flag']
X = X[['target_event', 'target_returns']]

start = '2024-01-01'
end = '2024-12-31'
X_test = X[(X.index >= start)&(X.index <= end)]
y_test = y[(y.index >= start)&(y.index <= end)]

X_with_const = sm.add_constant(X_test)

# OLS 회귀 적합
# ---------------------------------------------
lr = sm.OLS(y_test, X_with_const).fit(cov_type='HAC', cov_kwds={'maxlags': 10})
print("=== OLS with Newey-West Standard Errors ===")
print(lr.summary())

C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\1076028199.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['event_flag'] = X['event_flag'].shift(-3).fillna(0)


=== OLS with Newey-West Standard Errors ===
                            OLS Regression Results                            
Dep. Variable:                returns   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                     4.693
Date:                Mon, 07 Apr 2025   Prob (F-statistic):            0.00992
Time:                        16:58:06   Log-Likelihood:                 590.74
No. Observations:                 274   AIC:                            -1175.
Df Residuals:                     271   BIC:                            -1165.
Df Model:                           2                                         
Covariance Type:                  HAC                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------


C:\Users\slaye\AppData\Local\Temp\ipykernel_31360\1076028199.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['target_event'] = X['target'] * X['event_flag']
